# DV2607 Project Notebook
### Authors:
### Oliver Ljung (ollj19@student.bth.se)
### Phoebe Waters (phaa19@student.bth.se)

## Importing modules and dataset

In [ ]:
# imports
from matplotlib import pyplot as plt
import numpy as np

import tensorflow as tf

from keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, Flatten, Dense, Input, Activation, BatchNormalization, LeakyReLU, Reshape, UpSampling2D, Dropout, ReLU
from keras import Sequential, Model
from keras.datasets import mnist
from tensorflow.keras.datasets import cifar10
from keras.utils import to_categorical
import keras.backend as KB

from keras.losses import BinaryCrossentropy, CategoricalCrossentropy, Hinge, SquaredHinge, MeanSquaredError, Loss, SparseCategoricalCrossentropy

import random
import time

import art
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, CarliniL2Method, CarliniL0Method
from art.estimators.classification import KerasClassifier

tf.compat.v1.disable_eager_execution()      # Enable when training NN but has to be disabled for art
print(tf.config.list_physical_devices('GPU'))

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

## Defining functions

In [ ]:
# Defining functions

def display_attack(x_test, x_adv_test, model):
    x_real = x_test[:9]

    x_fake = x_adv_test[:9]
    x_fake_labels = model.predict(x_fake)
    x_real_labels = model.predict(x_real)

    for i in range(9):
        fig = plt.subplot(3, 3, i+1)
        fig.imshow(x_fake[i], cmap=plt.get_cmap('gray'))
        print(f'p_fake = {np.argmax(x_fake_labels[i])}, p_real = {np.argmax(x_real_labels[i])}')
    plt.show()

## Loading dataset

### MNIST

In [ ]:
# load dataset
(train_X, train_y), (test_X, test_y) = mnist.load_data()

train_X = train_X.astype("float32") / 255
test_X = test_X.astype("float32") / 255

train_X = np.expand_dims(train_X, -1)
test_X = np.expand_dims(test_X, -1)

train_y = to_categorical(train_y)
train_y = np.array([np.argmax(y) for y in train_y])
test_y  = to_categorical(test_y)
test_y = np.array([np.argmax(y) for y in test_y])

IMG_SHAPE = (28,28,1)
CHANNELS = 1
NUM_CLASSES = 10
IMG_LOW_LIMIT = 0
IMG_HIGH_LIMIT = 1

### CIFAR-10

In [ ]:
# load dataset
import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

# (train_X, train_y), (test_X, test_y) = cifar10.load_data()

# train_X = train_X.astype("float32")/255
# test_X = test_X.astype("float32")/255

# train_y = to_categorical(train_y)
# train_y = np.array([np.argmax(y) for y in train_y])
# test_y  = to_categorical(test_y)
# test_y = np.array([np.argmax(y) for y in test_y])

# IMG_SHAPE = (32,32,3)
# CHANNELS = 3
# NUM_CLASSES = 10
# IMG_LOW_LIMIT = 0
# IMG_HIGH_LIMIT = 1

## Load models

In [ ]:
# To use same models as in paper

model = tf.keras.models.load_model("models/mnist_classifier__report.h5")
discriminator = tf.keras.models.load_model("models/mnist_adv_discriminator_t0__report.h5")      
adv_generator = tf.keras.models.load_model("models/mnist_adv_generator_t0__report.h5")

# Have to convert classifier model to art
clf = KerasClassifier(model=model)

# Attacks

## 0. Baseline

In [ ]:
# Baseline for attack (% classified as 0s in test)
model.evaluate(test_X, np.zeros(len(test_X)))

## 1. AdvGAN

In [ ]:
perturbations = adv_generator.predict(test_X, verbose = 1)
advGAN_test_X = np.add(test_X, perturbations)
advGAN_test_X = np.clip(advGAN_test_X, 0, 1) # Values in image is [0,1]
model.evaluate(advGAN_test_X, np.zeros([len(advGAN_test_X), 1]))

In [ ]:
display_attack(test_X, advGAN_test_X, model)

## 2. FGM

In [ ]:
def fast_gradient_method_attack(x_test, clf):
    # MNIST params: eps=0.03, 10 steps
    # CIFAR-10 params: eps=0.01, 10 steps
    FGM = FastGradientMethod(clf, eps=0.03, targeted=True, batch_size=32)

    # Targeted Attack
    target_prediction = np.zeros([len(x_test), 1])

    FGM_test_X = FGM.generate(x_test, target_prediction)
    for step in range(10):
        FGM_test_X = FGM.generate(FGM_test_X, target_prediction)

    return FGM_test_X

FGM_test_X = fast_gradient_method_attack(test_X, clf)
model.evaluate(FGM_test_X, np.zeros(len(FGM_test_X)))

In [ ]:
display_attack(test_X, FGM_test_X, model)

## 2. PGD

In [ ]:
def projected_gradient_descent_attack(x_test, clf):
    # MNIST params: max_iter=10, eps_step=0.05, eps=0.3
    # CIFAR-10 params: max_iter=10, eps_step=0.01, eps=0.1, 
    PGD = ProjectedGradientDescent(clf, targeted=True, max_iter=10, eps_step=0.05, eps=0.3, verbose=False)

    # Targeted Attack
    target_prediction = np.zeros([len(x_test), 1])

    PGD_test_X = PGD.generate(x_test, target_prediction)

    return PGD_test_X

PGD_test_X = projected_gradient_descent_attack(test_X, clf)
model.evaluate(PGD_test_X, np.zeros(len(PGD_test_X)))

In [ ]:
display_attack(test_X, PGD_test_X, model)

# Defence

## 1. Binary Input detector (discriminator)

In [ ]:
class BinaryInputDetector():
    def __init__(self, detector, target_model):
        self._detector = detector
        self._target_model = target_model
    
    def predict(self, array):
        prediction = []
        
        for element in array:
            if self._detector.predict(np.array([element]))[0][0] < 0.5:
                # Advesary
                prediction.append(-1)
            else:
                # Real
                p = self._target_model.predict(np.array([element]))
                p_id = np.argmax(p)
                prediction.append(p_id)
        
        adv_rate = prediction.count(-1)/len(prediction)
        return np.array(prediction), adv_rate

In [ ]:
# discriminator = tf.keras.models.load_model("models/mnist_discriminator__report.h5")  # For normal GAN BID
BID = BinaryInputDetector(discriminator, model)

y_pred_BASE, adv_rate_BASE = BID.predict(test_X)
y_pred_FGM, adv_rate_FGM = BID.predict(FGM_test_X)
y_pred_PGD, adv_rate_PGD = BID.predict(PGD_test_X)
y_pred_advGAN , adv_rate_advGAN  = BID.predict(advGAN_test_X)

print(f"Adv rates: BASE: {adv_rate_BASE}, FGM: {adv_rate_FGM}, PGD: {adv_rate_PGD}, advGAN: {adv_rate_advGAN}")